# Image Classification for Coffee or Donuts

Is the given image a coffee photo, or a donut?
In this demo notebook we build a simple deep neural network to classify an image as one of the following:
- coffee
- mug
- donut


# Setup

In [2]:
credentials = {
  "bucket": "xdtest",
  "access_key_id": "da2ab693a5934be19ac1709d6c1c6367",
  "secret_access_key": "06daaa57b3fa8082af7620dd402c647e1a14e9ab9a4a7714",
  "endpoint_url": "https://s3.us.cloud-object-storage.appdomain.cloud"
}

## Install Prerequisites

We use Keras/Tensorflow to build the classification model, and visualize the process with matplotlib.

In [ ]:
!pip install tensorflow==1.15 ibm-cos-sdk==2.6

     |████████████████████████████████| 412.3MB 29kB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 3.8MB 28.5MB/s 
     |████████████████████████████████| 512kB 44.2MB/s 
     |████████████████████████████████| 768kB 48.7MB/s 
     |████████████████████████████████| 225kB 48.7MB/s 
     |████████████████████████████████| 552kB 43.6MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.6.0-py2.py3-none-any.whl size=72545 sha256=591b14b5a469a397fe92995315c0b0b0e54da6bc07a7c32156f48ed9b604b6d7
  Stored in directory: /root/.cache/pip/wheels/37/9c/c4/a2c610ccb877d37c2cb87a5bfe55845fecffd6bb01bcd5e9d5


In [ ]:
# Import required libraries
import os
import uuid
import shutil
import json
from botocore.client import Config
import ibm_boto3
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Read The Data

Here we build simple wrapper functions to read in the data from our cloud object storage buckets and extract it.

A download function from IBM Cloud

In [ ]:
def download_file_cos(credentials, local_file_name, key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=credentials['access_key_id'],
        aws_secret_access_key=credentials['secret_access_key'],
        endpoint_url=credentials['endpoint_url'])
    try:
        res=cos.download_file(Bucket=credentials['bucket'], Key=key, Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def get_annotations(credentials): 
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=credentials['access_key_id'],
        aws_secret_access_key=credentials['secret_access_key'],
        endpoint_url=credentials['endpoint_url'])
    try:
        return json.loads(cos.get_object(Bucket=credentials['bucket'], Key='_annotations.json')['Body'].read())
    except Exception as e:
        print(Exception, e)

In [ ]:
base_path = 'data'
if os.path.exists(base_path) and os.path.isdir(base_path):
    shutil.rmtree(base_path)
os.makedirs(base_path, exist_ok=True)

annotations = get_annotations(credentials)

for i, image in enumerate(annotations['annotations'].keys()):
    label = annotations['annotations'][image][0]['label']
    os.makedirs(os.path.join(base_path, label), exist_ok=True)
    _, extension = os.path.splitext(image)
    local_path = os.path.join(base_path, label, str(uuid.uuid4()) + extension)
    download_file_cos(credentials, local_path, image)

In [ ]:
!ls data/coffee
!ls data/donut
!ls data/mug

## Build the Model

We start with a [MobileNetV2](https://arxiv.org/abs/1801.04381) architecture as the backbone [pretrained feature extractor](https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet). We then add a couple of dense layers and a softmax layer to perfom the classification. We freeze the MobileNetV2 backbone with weights trained on ImageNet dataset and only train the dense layers and softmax layer that we have added.

In [ ]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(512,activation='relu')(x) #dense layer 1
x=tf.keras.layers.Dense(256,activation='relu')(x) #dense layer 2
preds=tf.keras.layers.Dense(3,activation='softmax')(x) #final layer with softmax activation

model=tf.keras.Model(inputs=base_model.input,outputs=preds)


In [ ]:
#Freeze layers from MobileNetV2 backbone (not to be trained)
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
#Prepare the training dataset as a data generator object
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('data',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='categorical',
                                                 shuffle=True)

### Using Adam, categorical_crossentropy and accuracy as optimization method, loss function and metrics, respectively

In [ ]:
# Build the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

## Train the model

In [ ]:
from tensorflow import set_random_seed
set_random_seed(2)
step_size_train=5
log_file = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=4)

## Figure of Training Loss and Accuracy

In [ ]:
# Model accuracy and loss vs epoch
plt.plot(log_file.history['acc'], '-bo', label="train_accuracy")
plt.plot(log_file.history['loss'], '-r*', label="train_loss")
plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend(loc='center right')
plt.show()

## Model Performance

Here we perform inference on some sample data points to determine the performance of the model

In [ ]:
# Mapping labels 
label_map = (train_generator.class_indices)

In [ ]:
label_map

In [ ]:
# Creating a sample inference function
def prediction(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    preds = model.predict(x)
    #print('Predictions', preds)
    
    for pred, value in label_map.items():    
        if value == np.argmax(preds):
            print('Predicted class is:', pred)
            print('With a confidence score of: ', np.max(preds))
    
    return np.argmax(preds)

In [ ]:
coffee_url = 'https://i5.walmartimages.com/asr/559143bb-3fd0-41b0-8358-fb0f74c41f8d_1.61dbeaff765619bbba67d4e519174932.jpeg'
mug_url = 'https://cdn.cnn.com/cnnnext/dam/assets/150929101049-black-coffee-stock-super-tease.jpg'
donut_url = 'https://i.ytimg.com/vi/gevpzxRxec4/maxresdefault.jpg'
!wget {coffee_url} -O Coffee.jpg 
!wget {mug_url} -O Mug.jpg
!wget {donut_url} -O Donut.jpg

In [ ]:
#Opening first image
image = Image.open("Coffee.jpg")
image

In [ ]:
#performing inference on above image
prediction('Coffee.jpg', model)

In [ ]:
#Opening second image
image = Image.open("Mug.jpg")
image

In [ ]:
prediction('Mug.jpg', model)

In [ ]:
#Opening third image
image = Image.open("Donut.jpg")
image

In [ ]:
prediction('Donut.jpg', model)